In [1]:
%ls -lah

total 20M
drwxr-xr-x 1 root root 4.0K May 12 20:08 ./
drwxr-xr-x 1 root root 4.0K May 12 16:48 ../
-rw-r--r-- 1 root root 3.2M May 12 16:56 19600101.txt
drwxr-xr-x 4 root root 4.0K May 11 16:34 .config/
drwxr-xr-x 2 root root 4.0K May 12 20:08 .ipynb_checkpoints/
drwxr-xr-x 1 root root 4.0K May 11 16:35 sample_data/
-rw-r--r-- 1 root root  17M May 12 18:37 temp.csv


In [2]:
!head temp.csv

datetime,datenum,data,year,month,lat
15-Jan-0000 12:00:00,15.5,277.519897460938,0,1,-82.5
15-Jan-0000 12:00:00,15.5,277.413146972656,0,1,-67.5
15-Jan-0000 12:00:00,15.5,277.559020996094,0,1,-52.5
15-Jan-0000 12:00:00,15.5,277.692993164062,0,1,-37.5
15-Jan-0000 12:00:00,15.5,277.851440429688,0,1,-22.5
15-Jan-0000 12:00:00,15.5,277.736114501953,0,1,-7.5
15-Jan-0000 12:00:00,15.5,277.867553710938,0,1,7.5
15-Jan-0000 12:00:00,15.5,278.610809326172,0,1,22.5
15-Jan-0000 12:00:00,15.5,279.152557373047,0,1,37.5


In [3]:
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
lon = []
for x in range(-180, 182, 2):
  lon.append([float(x)])

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark import SparkContext
from pyspark.sql import Row
from datetime import datetime
import reverse_geocoder as rg 
from pyspark.sql.functions import udf
from pyspark.sql.functions import countDistinct

def reverseGeocode(c1, c2):
    result = rg.search([c1, c2])
    return result[0]['cc']

In [ ]:
sc = SparkContext.getOrCreate()
rdd = sc.textFile('co2_csv_data.csv')
header = rdd.first()
rdd = rdd.filter(lambda line: line != header)

longit = []
for x in range(-180, 180, 2):
  longit.append(x)

# Extract the desired columns
rdd_final = rdd.map(lambda line: line.split(','))\
  .map(lambda row: (int(row[3]), int(row[4]), float(row[2]), float(row[5])))\
  .filter(lambda row: row[0] >= 1960)\
  .flatMap(lambda row: [(row[0],row[1],row[2],row[3], x) for x in longit])\
  .map(lambda row: ( (row[0], row[1],reverseGeocode(row[3], row[4])),  row[2]))\
  .reduceByKey(lambda x, y: x)

for row in rdd_final.take(10):
    print(row)

In [ ]:
save_rdd = rdd_final.map(lambda row: ','.join(map(str, row)))
save_rdd.saveAsTextFile('big_data_co2_country_monthly.csv')